In [4]:
import pandas as pd

customers = pd.read_csv("../data/customers.csv")
subscriptions = pd.read_csv("../data/subscriptions.csv")
user_activity = pd.read_csv("../data/user_activity.csv")
transactions = pd.read_csv("../data/transactions.csv")

In [5]:
print("customers:", customers.shape)
print("subscriptions:", subscriptions.shape)
print("user_activity:", user_activity.shape)
print("transactions:", transactions.shape)

customers.head(3)

customers: (50000, 5)
subscriptions: (100000, 6)
user_activity: (1000000, 4)
transactions: (500000, 4)


,customer_id,name,email,signup_date,country
0,1,Tara Ramos,pinedajill@gmail.com,2025-02-09,Bermuda
1,2,Casey Sanders,kimberly26@white-velasquez.biz,2022-10-11,Lao People's Democratic Republic
2,3,Amanda Fernandez,annapugh@wiggins.biz,2023-04-19,Sweden


In [6]:
# Make all column names lowercase and strip spaces
def clean_columns(df):
    df.columns = [c.strip().lower() for c in df.columns]
    return df

customers = clean_columns(customers)
subscriptions = clean_columns(subscriptions)
user_activity = clean_columns(user_activity)
transactions = clean_columns(transactions)

# Check columns
print(customers.columns)
print(subscriptions.columns)
print(user_activity.columns)
print(transactions.columns)


Index(['customer_id', 'name', 'email', 'signup_date', 'country'], dtype='object')
Index(['subscription_id', 'customer_id', 'plan_type', 'start_date', 'end_date',
       'status'],
      dtype='object')
Index(['activity_id', 'customer_id', 'event_type', 'event_date'], dtype='object')
Index(['transaction_id', 'customer_id', 'amount', 'transaction_date'], dtype='object')


In [7]:
# Convert date columns to datetime
customers["signup_date"] = pd.to_datetime(customers["signup_date"], errors="coerce")
subscriptions["start_date"] = pd.to_datetime(subscriptions["start_date"], errors="coerce")
subscriptions["end_date"] = pd.to_datetime(subscriptions["end_date"], errors="coerce")
user_activity["event_date"] = pd.to_datetime(user_activity["event_date"], errors="coerce")
transactions["transaction_date"] = pd.to_datetime(transactions["transaction_date"], errors="coerce")

# Quick check
print(customers.dtypes)
print(subscriptions.dtypes)
print(user_activity.dtypes)
print(transactions.dtypes)

customer_id             int64
name                   object
email                  object
signup_date    datetime64[ns]
country                object
dtype: object
subscription_id             int64
customer_id                 int64
plan_type                  object
start_date         datetime64[ns]
end_date           datetime64[ns]
status                     object
dtype: object
activity_id             int64
customer_id             int64
event_type             object
event_date     datetime64[ns]
dtype: object
transaction_id               int64
customer_id                  int64
amount                     float64
transaction_date    datetime64[ns]
dtype: object


In [8]:
# Missing value percentage per column (top offenders)
def missing_summary(df, name):
    print(f"\n{name} — missing % (top 10):")
    display(
        (df.isna().mean() * 100)
        .sort_values(ascending=False)
        .head(10)
        .to_frame("missing_pct")
    )

missing_summary(customers, "customers")
missing_summary(subscriptions, "subscriptions")
missing_summary(user_activity, "user_activity")
missing_summary(transactions, "transactions")


customers — missing % (top 10):


,missing_pct
customer_id,0.0
name,0.0
email,0.0
signup_date,0.0
country,0.0



subscriptions — missing % (top 10):


,missing_pct
end_date,30.034
subscription_id,0.000
customer_id,0.000
plan_type,0.000
start_date,0.000
status,0.000



user_activity — missing % (top 10):


,missing_pct
activity_id,0.0
customer_id,0.0
event_type,0.0
event_date,0.0



transactions — missing % (top 10):


,missing_pct
transaction_id,0.0
customer_id,0.0
amount,0.0
transaction_date,0.0


In [9]:
print("Customers table — unique customer_ids:", customers["customer_id"].nunique())
print("Subscriptions table — unique customer_ids:", subscriptions["customer_id"].nunique())
print("User activity table — unique customer_ids:", user_activity["customer_id"].nunique())
print("Transactions table — unique customer_ids:", transactions["customer_id"].nunique())

Customers table — unique customer_ids: 50000
Subscriptions table — unique customer_ids: 43181
User activity table — unique customer_ids: 50000
Transactions table — unique customer_ids: 49998


In [10]:
print("Signup date range:", customers["signup_date"].min(), "→", customers["signup_date"].max())
print("Transaction date range:", transactions["transaction_date"].min(), "→", transactions["transaction_date"].max())
print("User activity date range:", user_activity["event_date"].min(), "→", user_activity["event_date"].max())

Signup date range: 2022-03-10 00:00:00 → 2025-03-09 00:00:00
Transaction date range: 2022-03-10 00:00:00 → 2025-03-09 00:00:00
User activity date range: 2022-03-10 00:00:00 → 2025-03-09 00:00:00


In [11]:
# Determine a safe analysis date (90 days before last transaction)
last_transaction_date = transactions["transaction_date"].max()
analysis_date = last_transaction_date - pd.Timedelta(days=90)

print("Last transaction date:", last_transaction_date)
print("Analysis date:", analysis_date)

Last transaction date: 2025-03-09 00:00:00
Analysis date: 2024-12-09 00:00:00


In [12]:
# Define forward-looking churn window
label_start = analysis_date + pd.Timedelta(days=1)
label_end = last_transaction_date

# Transactions in the next 90 days
future_txns = transactions[
    (transactions["transaction_date"] >= label_start) &
    (transactions["transaction_date"] <= label_end)
]

print("Label window:", label_start, "→", label_end)
print("Rows in future_txns:", future_txns.shape[0])

future_txns.head()

Label window: 2024-12-10 00:00:00 → 2025-03-09 00:00:00
Rows in future_txns: 41080


,transaction_id,customer_id,amount,transaction_date
6,7,3635,26.24,2025-02-25
51,52,11607,185.96,2025-01-02
52,53,27826,151.22,2025-01-09
58,59,13508,70.89,2024-12-28
63,64,10385,35.02,2025-02-28


In [13]:
# Count future transactions per customer
future_txn_counts = (
    future_txns
    .groupby("customer_id")
    .size()
    .reset_index(name="future_txn_count")
)

future_txn_counts.head()

,customer_id,future_txn_count
0,3,1
1,4,1
2,7,4
3,9,1
4,10,3


In [14]:
# Start from full customer list
labels = customers[["customer_id"]].copy()

# Merge future transaction counts
labels = labels.merge(
    future_txn_counts,
    on="customer_id",
    how="left"
)

# Customers with no future transactions → churn
labels["future_txn_count"] = labels["future_txn_count"].fillna(0).astype(int)
labels["churn_next_90d"] = (labels["future_txn_count"] == 0).astype(int)

labels.head()

,customer_id,future_txn_count,churn_next_90d
0,1,0,1
1,2,0,1
2,3,1,0
3,4,1,0
4,5,0,1


In [15]:
print("Total customers:", labels.shape[0])
print("Churned customers:", labels["churn_next_90d"].sum())
print("Churn rate:", labels["churn_next_90d"].mean())

Total customers: 50000
Churned customers: 21827
Churn rate: 0.43654


In [16]:
# Define past windows
past_30_start = analysis_date - pd.Timedelta(days=30)
past_90_start = analysis_date - pd.Timedelta(days=90)

# Transactions BEFORE analysis date
txn_past = transactions[transactions["transaction_date"] <= analysis_date]

# 30-day window
txn_30 = txn_past[txn_past["transaction_date"] > past_30_start]

# 90-day window
txn_90 = txn_past[txn_past["transaction_date"] > past_90_start]

# Aggregate transaction features
txn_features = labels[["customer_id"]].copy()

txn_features = txn_features.merge(
    txn_30.groupby("customer_id").agg(
        txn_count_30d=("transaction_id", "count"),
        txn_amount_sum_30d=("amount", "sum"),
        txn_amount_avg_30d=("amount", "mean")
    ).reset_index(),
    on="customer_id",
    how="left"
)

txn_features = txn_features.merge(
    txn_90.groupby("customer_id").agg(
        txn_count_90d=("transaction_id", "count"),
        txn_amount_sum_90d=("amount", "sum"),
        txn_amount_avg_90d=("amount", "mean")
    ).reset_index(),
    on="customer_id",
    how="left"
)

txn_features.head()

,customer_id,txn_count_30d,txn_amount_sum_30d,txn_amount_avg_30d,txn_count_90d,txn_amount_sum_90d,txn_amount_avg_90d
0,1,NaN,NaN,NaN,1.0,106.98,106.98
1,2,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,1.0,185.33,185.33
3,4,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
print("Rows:", txn_features.shape[0])
print(txn_features.isna().sum())

Rows: 50000
customer_id               0
txn_count_30d         38079
txn_amount_sum_30d    38079
txn_amount_avg_30d    38079
txn_count_90d         22078
txn_amount_sum_90d    22078
txn_amount_avg_90d    22078
dtype: int64


In [18]:
# User activity BEFORE analysis date
ua_past = user_activity[user_activity["event_date"] <= analysis_date]

# Past windows
ua_30 = ua_past[ua_past["event_date"] > past_30_start]
ua_90 = ua_past[ua_past["event_date"] > past_90_start]

# Focus on logins only
login_30 = ua_30[ua_30["event_type"] == "Login"]
login_90 = ua_90[ua_90["event_type"] == "Login"]

login_30.head()

,activity_id,customer_id,event_type,event_date
77,78,21048,Login,2024-11-25
111,112,29268,Login,2024-12-05
341,342,15458,Login,2024-11-28
345,346,19372,Login,2024-12-04
399,400,12457,Login,2024-12-04


In [19]:
# Start activity feature table
ua_features = labels[["customer_id"]].copy()

# Login counts
ua_features = ua_features.merge(
    login_30.groupby("customer_id").size().rename("logins_30d").reset_index(),
    on="customer_id",
    how="left"
)

ua_features = ua_features.merge(
    login_90.groupby("customer_id").size().rename("logins_90d").reset_index(),
    on="customer_id",
    how="left"
)

# Last login date as of analysis date
last_login = (
    ua_past[ua_past["event_type"] == "Login"]
    .groupby("customer_id")["event_date"]
    .max()
    .reset_index(name="last_login_date")
)

ua_features = ua_features.merge(
    last_login,
    on="customer_id",
    how="left"
)

# Days since last login
ua_features["days_since_last_login"] = (
    analysis_date - ua_features["last_login_date"]
).dt.days

ua_features.head()

,customer_id,logins_30d,logins_90d,last_login_date,days_since_last_login
0,1,NaN,NaN,2024-02-02,311.0
1,2,NaN,NaN,2024-05-28,195.0
2,3,NaN,1.0,2024-10-24,46.0
3,4,NaN,NaN,2024-03-19,265.0
4,5,NaN,NaN,2024-03-01,283.0


In [20]:
print("Rows:", ua_features.shape[0])
print(ua_features.isna().sum())

Rows: 50000
customer_id                  0
logins_30d               40239
logins_90d               26052
last_login_date             35
days_since_last_login       35
dtype: int64


In [23]:
# Remove customers who signed up after the analysis date
cust_features = cust_features[cust_features["signup_date"] <= analysis_date].copy()

# Recalculate account age just to be safe
cust_features["account_age_days"] = (
    analysis_date - cust_features["signup_date"]
).dt.days

cust_features.head()


,customer_id,signup_date,account_age_days
1,2,2022-10-11,790
2,3,2023-04-19,600
3,4,2024-05-05,218
4,5,2023-12-25,350
5,6,2022-05-07,947


In [24]:
print("Rows after filter:", cust_features.shape[0])
print("Any negative ages?", (cust_features["account_age_days"] < 0).sum())

Rows after filter: 46013
Any negative ages? 0


In [25]:
# Keep only customers who existed as of analysis_date
valid_customers = cust_features[["customer_id"]].copy()

labels_final = labels.merge(
    valid_customers,
    on="customer_id",
    how="inner"
)

print("Labels before filter:", labels.shape[0])
print("Labels after filter:", labels_final.shape[0])

Labels before filter: 50000
Labels after filter: 46013


In [26]:
# Start from filtered labels
model_df = labels_final.copy()

# Merge transaction features
model_df = model_df.merge(
    txn_features,
    on="customer_id",
    how="left"
)

# Merge user activity features
model_df = model_df.merge(
    ua_features,
    on="customer_id",
    how="left"
)

# Merge customer profile features
model_df = model_df.merge(
    cust_features[["customer_id", "account_age_days"]],
    on="customer_id",
    how="left"
)

model_df.head()

,customer_id,future_txn_count,churn_next_90d,txn_count_30d,txn_amount_sum_30d,txn_amount_avg_30d,txn_count_90d,txn_amount_sum_90d,txn_amount_avg_90d,logins_30d,logins_90d,last_login_date,days_since_last_login,account_age_days
0,2,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-28,195.0,790
1,3,1,0,NaN,NaN,NaN,1.0,185.33,185.33,NaN,1.0,2024-10-24,46.0,600
2,4,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-19,265.0,218
3,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-01,283.0,350
4,6,0,1,1.0,49.11,49.11,1.0,49.11,49.11,NaN,NaN,2024-08-04,127.0,947


In [27]:
print("Rows:", model_df.shape[0])
print("Unique customers:", model_df["customer_id"].nunique())

print("\nMissing values per column:")
print(model_df.isna().sum().sort_values(ascending=False))

Rows: 46013
Unique customers: 46013

Missing values per column:
logins_30d               37017
txn_amount_sum_30d       35064
txn_amount_avg_30d       35064
txn_count_30d            35064
logins_90d               23973
txn_amount_sum_90d       20366
txn_amount_avg_90d       20366
txn_count_90d            20366
days_since_last_login       31
last_login_date             31
churn_next_90d               0
future_txn_count             0
customer_id                  0
account_age_days             0
dtype: int64


In [28]:
# Copy to avoid mutating the original
model_ready = model_df.copy()

# Drop raw date column
model_ready = model_ready.drop(columns=["last_login_date"])

# Fill transaction-related NaNs with 0
txn_cols = [
    "txn_count_30d", "txn_amount_sum_30d", "txn_amount_avg_30d",
    "txn_count_90d", "txn_amount_sum_90d", "txn_amount_avg_90d"
]

model_ready[txn_cols] = model_ready[txn_cols].fillna(0)

# Fill login counts with 0
model_ready["logins_30d"] = model_ready["logins_30d"].fillna(0)
model_ready["logins_90d"] = model_ready["logins_90d"].fillna(0)

# Days since last login: large number for "never"
model_ready["days_since_last_login"] = model_ready["days_since_last_login"].fillna(9999)

# Final check
print(model_ready.isna().sum())
model_ready.head()


customer_id              0
future_txn_count         0
churn_next_90d           0
txn_count_30d            0
txn_amount_sum_30d       0
txn_amount_avg_30d       0
txn_count_90d            0
txn_amount_sum_90d       0
txn_amount_avg_90d       0
logins_30d               0
logins_90d               0
days_since_last_login    0
account_age_days         0
dtype: int64


,customer_id,future_txn_count,churn_next_90d,txn_count_30d,txn_amount_sum_30d,txn_amount_avg_30d,txn_count_90d,txn_amount_sum_90d,txn_amount_avg_90d,logins_30d,logins_90d,days_since_last_login,account_age_days
0,2,0,1,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,195.0,790
1,3,1,0,0.0,0.00,0.00,1.0,185.33,185.33,0.0,1.0,46.0,600
2,4,1,0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,265.0,218
3,5,0,1,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,283.0,350
4,6,0,1,1.0,49.11,49.11,1.0,49.11,49.11,0.0,0.0,127.0,947


In [29]:
# Define target
y = model_ready["churn_next_90d"]

# Define features (drop identifiers and target)
X = model_ready.drop(columns=["customer_id", "churn_next_90d"])

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (46013, 11)
y shape: (46013,)


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

print("\nChurn rate (overall):", y.mean())
print("Churn rate (train):", y_train.mean())
print("Churn rate (test):", y_test.mean())

Train size: 36810
Test size: 9203

Churn rate (overall): 0.43674613696129355
Churn rate (train): 0.4367563162184189
Churn rate (test): 0.4367054221449527


In [31]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Build pipeline
logreg_pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=1000, random_state=42))
])

# Train
logreg_pipeline.fit(X_train, y_train)

print("Logistic Regression trained.")

Logistic Regression trained.


In [32]:
from sklearn.metrics import confusion_matrix, classification_report

# Predictions
y_pred = logreg_pipeline.predict(X_test)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[5184    0]
 [   0 4019]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5184
           1       1.00      1.00      1.00      4019

    accuracy                           1.00      9203
   macro avg       1.00      1.00      1.00      9203
weighted avg       1.00      1.00      1.00      9203



In [33]:
from sklearn.metrics import roc_auc_score

# Predicted probabilities for churn = 1
y_prob = logreg_pipeline.predict_proba(X_test)[:, 1]

roc_auc = roc_auc_score(y_test, y_prob)
print("ROC-AUC:", roc_auc)

ROC-AUC: 1.0


In [35]:
# Train using leakage-free features
logreg_pipeline.fit(
    X_noleak.loc[X_train.index],
    y_train
)

# Predict on test set
y_pred_noleak = logreg_pipeline.predict(
    X_noleak.loc[X_test.index]
)

y_prob_noleak = logreg_pipeline.predict_proba(
    X_noleak.loc[X_test.index]
)[:, 1]

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_noleak))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_noleak))

print("\nROC-AUC:", roc_auc_score(y_test, y_prob_noleak))


Confusion Matrix:
[[5184    0]
 [4019    0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.56      1.00      0.72      5184
           1       0.00      0.00      0.00      4019

    accuracy                           0.56      9203
   macro avg       0.28      0.50      0.36      9203
weighted avg       0.32      0.56      0.41      9203


ROC-AUC: 0.5003513403923954


C:\Users\faith\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\faith\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\faith\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [36]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# Rebuild pipeline with class weights
logreg_balanced = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(
        max_iter=1000,
        random_state=42,
        class_weight="balanced"
    ))
])

# Train
logreg_balanced.fit(
    X_noleak.loc[X_train.index],
    y_train
)

# Predict
y_pred_bal = logreg_balanced.predict(X_noleak.loc[X_test.index])
y_prob_bal = logreg_balanced.predict_proba(X_noleak.loc[X_test.index])[:, 1]

# Evaluate
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_bal))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_bal))

print("\nROC-AUC:", roc_auc_score(y_test, y_prob_bal))


Confusion Matrix:
[[2448 2736]
 [1879 2140]]

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.47      0.51      5184
           1       0.44      0.53      0.48      4019

    accuracy                           0.50      9203
   macro avg       0.50      0.50      0.50      9203
weighted avg       0.51      0.50      0.50      9203


ROC-AUC: 0.5006170055661534


In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_leaf=50,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_noleak.loc[X_train.index], y_train)

y_pred_rf = rf.predict(X_noleak.loc[X_test.index])
y_prob_rf = rf.predict_proba(X_noleak.loc[X_test.index])[:, 1]

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

print("\nROC-AUC:", roc_auc_score(y_test, y_prob_rf))


Confusion Matrix:
[[5139   45]
 [3989   30]]

Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.99      0.72      5184
           1       0.40      0.01      0.01      4019

    accuracy                           0.56      9203
   macro avg       0.48      0.50      0.37      9203
weighted avg       0.49      0.56      0.41      9203


ROC-AUC: 0.5040082323085713
